In [1]:
import torch
import torch.nn as nn

In [32]:
import os
import torch
import torch.nn as nn

class OC(nn.Module):
    def __init__(self,
                 seqlen=16
                 ):
        super().__init__()
        self.seqlen = seqlen
        self.conv1 = nn.Sequential(
            nn.Linear(seqlen, seqlen),
            nn.LayerNorm(seqlen),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Linear(seqlen//2, seqlen//2),
            nn.LayerNorm(seqlen//2),
            nn.ReLU()
        )
        self.conv3 = nn.Sequential(
            nn.Linear(seqlen//4, seqlen//4),
            nn.LayerNorm(seqlen//4),
            nn.ReLU()
        )
 
    def forward(self, x, is_train=False, J_regressor=None):
        B = x.shape[0]
        init_x = x
        x = x.permute(0, 2, 1)
        x = self.conv1(x)                               # [B, 2048, 16]
        x = x[..., self.seqlen//2-4 : self.seqlen//2+4] # [B, 2048, 8]
        x = self.conv2(x)                               # [B, 2048, 8]
        x = x[..., self.seqlen//4-2 : self.seqlen//4+2] # [B, 2048, 4]
        x = self.conv3(x)
        x = x[..., self.seqlen//8-1: self.seqlen//8+1]    # [B, ]
        
        x = x.permute(0, 2, 1)  
        x = torch.mean(x, dim=1, keepdim=True)

x = torch.rand((4, 16, 2048))
model = OC()
model(x)      


torch.Size([4, 1, 2048])
